<a href="https://colab.research.google.com/github/afrago/dl-upna-face-recognition/blob/master/dl_upna_Face_Recognition_03_VGGFace2Keras_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconocimiento Facial - VGGFace2 - Keras
## Comparación entre los diferentes modelos ofrecidos 

La biblioteca keras-vggface proporciona tres modelos VGGModels pre-entrenados, un modelo VGGFace1 a través de model='vgg16′ (el predeterminado), y dos modelos VGGFace2 'resnet50' y 'senet50'.

## Instalar la biblioteca keras-vggface 


In [0]:
! pip install matplotlib
! pip install git+https://github.com/rcmalli/keras-vggface.git
! pip install mtcnn


  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-3zs4t85f
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-3zs4t85f
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=e6de193db2bf4b12e10557049ef44c2c790a765a65aa9fd68af39f7791f3515e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ofbepbo1/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
     |████████████████████████████████| 2.3MB 2.8MB/s 


## Detector facial

In [0]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = np.array(image)
	return face_array

### VGG16

VGG16 es un modelo de red neural convolucional propuesto por K. Simonyan y A. Zisserman de la Universidad de Oxford en el documento "Redes convolucionales muy profundas para el reconocimiento de imágenes a gran escala". El modelo alcanza una precisión del 92,7% en la prueba Top-5 de ImageNet, que es un conjunto de datos de más de 14 millones de imágenes pertenecientes a 1000 clases.

[Fuente](https://neurohive.io/en/popular-networks/vgg16/)

![Arquitectura](https://neurohive.io/wp-content/uploads/2018/11/vgg16-neural-network.jpg)

In [0]:
import matplotlib.pyplot as pyplot
from mtcnn.mtcnn import MTCNN
from PIL import Image
import numpy as np
from numpy import expand_dims
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import decode_predictions

!wget "https://upload.wikimedia.org/wikipedia/commons/6/6e/Sharon_Stone_Cannes_2013_2.jpg" -O "sharon_stone1.jpg"

# load the photo and extract the face
pixels = extract_face('sharon_stone1.jpg')
# convert one face into samples
pixels = pixels.astype('float32')
samples = expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = preprocess_input(samples, version=1)
# create a vggface model
model = VGGFace(model='vgg16')
# perform prediction
yhat = model.predict(samples)

model.summary()

# convert prediction into names
results = decode_predictions(yhat)
# display most likely results
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

Using TensorFlow backend.


--2020-05-15 08:45:46--  https://upload.wikimedia.org/wikipedia/commons/6/6e/Sharon_Stone_Cannes_2013_2.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82711 (81K) [image/jpeg]
Saving to: ‘sharon_stone1.jpg’

sharon_stone1.jpg   100%[===================>]  80.77K  --.-KB/s    in 0.1s    

2020-05-15 08:45:47 (670 KB/s) - ‘sharon_stone1.jpg’ saved [82711/82711]

580075520/580070376 [==============================] - 36s 0us/step
Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_____________

## RESNET50

In [0]:
# load the photo and extract the face
pixels = extract_face('sharon_stone1.jpg')
# convert one face into samples
pixels = pixels.astype('float32')
samples = expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = preprocess_input(samples, version=2)
# create a vggface model
model = VGGFace(model='resnet50')
# perform prediction
yhat = model.predict(samples)

model.summary()

# convert prediction into names
results = decode_predictions(yhat)
# display most likely results
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

165445632/165439116 [==============================] - 11s 0us/step
Model: "vggface_resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_8[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
_______________

## SENET50

Las redes neuronales convolucionales se construyen sobre la operación de convolución, que extrae características informativas de
fusionando información espacial y de canales en el interior de
campos receptivos locales. Para aumentar el poder de representación de una red, varios enfoques recientes han
mostró el beneficio de mejorar la codificación espacial. En este
trabajo, nos centramos en la relación con el canal y proponemos
una novedosa unidad arquitectónica, que llamamos el bloque "Aprieta-y-Excita" (SE), que recalibra de forma adaptativa
respuestas de las características de los canales mediante la modelización explícita de las interdependencias entre los canales. Demostramos que
apilando estos bloques juntos, podemos construir arquitecturas SENet que se generalizan muy bien a través de desafiantes conjuntos de datos. Crucialmente, encontramos que los bloques SE producen
mejoras significativas en el rendimiento de las arquitecturas profundas de última generación existentes con un mínimo costo adicional de computación. Los SENets formaron la base de nuestro ILSVRC
La presentación de la clasificación de 2017 que ganó el primer lugar y
redujo significativamente el error de los cinco primeros al 2,251%, logrando una mejora relativa del 10% con respecto a la entrada ganadora de 2016. El código y los modelos están disponibles en https:
//github.com/hujie-frank/SENet.

[Arquitectura](https://github.com/hujie-frank/SENet#approach)

In [0]:
# load the photo and extract the face
pixels = extract_face('sharon_stone1.jpg')
# convert one face into samples
pixels = pixels.astype('float32')
samples = expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = preprocess_input(samples, version=2)
# create a vggface model
model = VGGFace(model='senet50')
# perform prediction
yhat = model.predict(samples)



# convert prediction into names
results = decode_predictions(yhat, )
# display most likely results
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

b' Sharon_Stone': 99.784%
b' Tina_Maze': 0.080%
b' Federica_Fontana': 0.011%
b' Noelle_Reno': 0.009%
b' Emma_Atkins': 0.008%
